# **Homework Assignment #1**

Assigned: January 10, 2022

Due: January 24, 2022



---

This assignment consists of four questions that require a short answer and one that requires you to generate some Python code. You can enter your answers and your code directly in a Colaboratory notebook and upload the **shareable** link for your notebook as your homework submission.

You must complete this assignment individually; you are not allowed to collaborate with anyone
else. You may discuss the homework to understand the problems and the mathematics behind the methods, but you are not allowed to share problem solutions or your code with any other students. Similarly, you are also not allowed to consult the internet for solutions.

---

#1.

(15 points) Consider a scenario in which you train a decision tree to classify credit card transactions into fraudulent (+) or legitimate (-). You have a dataset with 10,000 labeled transactions (5,000 + and 5,000 -). You randomly select 8,000 examples to build a decision tree with a minimum leaf size of 2 data points, then test your tree on the held-out 2,000 data points. You observe that your training error is 0.01 but your test error is 0.20. Explain why this might happen and the steps that you could take to improve the performance of the decision tree algorithm.

#2.

(25 points) Supreme Leader Snoke of the First Order has been concerned about the number of stormtroopers who have displayed aberrant behavior such as dereliction of duty on the battlefield lately, so Kylo Ren has asked Captain Phasma to build a decision tree to predict whether behavior is an Abberation (the target variable) based on the features Base (Starkiller or Snoke), Clone (Yes or No), Veteran (Yes or No), and Commander (Phasma, Inky, Blinky, or Clyde). Use the Information Gain criteria described in class to build a decision tree based on the training data below. Show each calculation you perform to generate the tree. You may upload the solution as a tree picture or describe the nodes and edges at each level of the tree.

ID | Base | Clone | Veteran | Commander | Aberration
--- | --- | --- | --- | --- | ---
1 | Starkiller | No | No | Phasma | No
2 | Starkiller | No | No | Phasma | No
3 | Starkiller | No | Yes | Inky | No
4 | Snoke | No | Yes | Inky | Yes
5 | Snoke | No | No | Phasma | Yes
6 | Snoke | Yes | Yes | Inky | No
7 | Starkiller | Yes | No | Blinky | No
8 | Starkiller | Yes | Yes | Blinky |Yes
9 | Starkiller | Yes | Yes | Blinky | Yes
10 | Starkiller | Yes | No | Blinky | No
11 | Snoke | No | Yes | Clyde | No
12 | Snoke | Yes | No | Clyde | No
13 | Snoke | No | Yes | Clyde | No
14 | Snoke | Yes | Yes | Clyde | No

#3.

(10 points) Express the concept (Aberration=Yes) learned by your trees in Problem 1 as logical if-then rules.

#4.

(10 points) Suppose you are testing a new algorithm on a data set consisting of 100 positive and 100 negative examples. You plan to use leave-one-out cross-validation and compare your algorithm to a baseline function, a simple majority classifier. With leave-one-out cross-validation, you train the algorithm on 199 data points and test it on 1 data point. You repeat the process 200 times, letting each point having a chance to represent the test set, and report the average of the classification accuracies. Given a set of training data, the majority classifier always outputs the class that is in the majority in the training set, regardless of the input. You expect the majority classifier to achieve about 50% classification accuracy, but to your surprise, it scores zero every time. Why?

#5.

(60 points) In this problem you are asked to write a Python program using Google Colab. We provide code below to construct a decision tree using the measures of entropy and gain discussed in class. You need to add three steps.

- First, implement a majority classifier as we described in class and update the code to use either the majority classifier or the decision tree on the dataset.

- Second, implement a "random choice" classifier that picks a class label based on a uniform probability distribution and update the code to use any of the three alternative classifiers.

- Third, update the code to run each classifier 10 times and average the accuracy for each classifier over the 10 runs. Each run should randomly pick 7 training data points from the set and use the remaining 3 points for classification. Report the average accuracies for your three methods.

*Note that all of the code you write needs to be entirely your own, not copied from another existing program or using existing libraries that perform the specified functionality.*

In [ ]:
# Decision tree learning
#
# Assumes discrete features. Examples may be inconsistent. Stopping condition for tree
# generation is when all examples have the same class, or there are no more features
# to split on (in which case, use the majority class). If a split yields no examples
# for a particular feature value, then the classification is based on the parent's
# majority class.

import math

class TreeNode:
    def __init__(self, majClass):
        self.split_feature = -1 # -1 indicates leaf node
        self.children = {} # dictionary of {feature_value: child_tree_node}
        self.majority_class = majClass
        
def build_tree(examples):
    if not examples:
        return None
    # collect sets of values for each feature index, based on the examples
    features = {}
    for feature_index in range(len(examples[0]) - 1):
        features[feature_index] = set([example[feature_index] for example in examples])
    return build_tree_1(examples, features)
    
def build_tree_1(examples, features):
    tree_node = TreeNode(majority_class(examples))
    # if examples all have same class, then return leaf node predicting this class
    if same_class(examples):
        return tree_node
    # if no more features to split on, then return leaf node predicting majority class
    if not features:
        return tree_node
    # split on best feature and recursively generate children
    best_feature_index = best_feature(features, examples)
    tree_node.split_feature = best_feature_index
    remaining_features = features.copy()
    remaining_features.pop(best_feature_index)
    for feature_value in features[best_feature_index]:
        split_examples = filter_examples(examples, best_feature_index, feature_value)
        tree_node.children[feature_value] = build_tree_1(split_examples, remaining_features)
    return tree_node

def majority_class(examples):
    classes = [example[-1] for example in examples]
    return max(set(classes), key = classes.count)

def same_class(examples):
    classes = [example[-1] for example in examples]
    return (len(set(classes)) == 1)

def best_feature(features, examples):
    # Return index of feature with lowest entropy after split
    best_feature_index = -1
    best_entropy = 2.0 # max entropy = 1.0
    for feature_index in features:
        se = split_entropy(feature_index, features, examples)
        if se < best_entropy:
            best_entropy = se
            best_feature_index = feature_index
    return best_feature_index

def split_entropy(feature_index, features, examples):
    # Return weighted sum of entropy of each subset of examples by feature value.
    se = 0.0
    for feature_value in features[feature_index]:
        split_examples = filter_examples(examples, feature_index, feature_value)
        se += (float(len(split_examples)) / float(len(examples))) * entropy(split_examples)
    return se

def entropy(examples):
    classes = [example[-1] for example in examples]
    classes_set = set(classes)
    class_counts = [classes.count(c) for c in classes_set]
    e = 0.0
    class_sum = sum(class_counts)
    for class_count in class_counts:
        if class_count > 0:
            class_frac = float(class_count) / float(class_sum)
            e += (-1.0)* class_frac * math.log(class_frac, 2.0)
    return e

def filter_examples(examples, feature_index, feature_value):
    # Return subset of examples with given value for given feature index.
    return list(filter(lambda example: example[feature_index] == feature_value, examples))

def print_tree(tree_node, feature_names, depth = 1):
    indent_space = depth * "  "
    if tree_node.split_feature == -1: # leaf node
        print(indent_space + feature_names[-1] + ": " + tree_node.majority_class)
    else:
        for feature_value in tree_node.children:
            print(indent_space + feature_names[tree_node.split_feature] + " == " + feature_value)
            child_node = tree_node.children[feature_value]
            if child_node:
                print_tree(child_node, feature_names, depth+1)
            else:
                # no child node for this value, so use majority class of parent (tree_node)
                print(indent_space + "  " + feature_names[-1] + ": " + tree_node.majority_class)

def classify(tree_node, instance):
    if tree_node.split_feature == -1:
        return tree_node.majority_class
    child_node = tree_node.children[instance[tree_node.split_feature]]
    if child_node:
        return classify(child_node, instance)
    else:
        return tree_node.majority_class

if __name__ == "__main__":
   feature_names = ["Color", "Type", "Origin", "Stolen"]
   
   examples = [
       ["Red", "Sports", "Domestic", "Yes"],
       ["Red", "Sports", "Domestic", "No"],
       ["Red", "Sports", "Domestic", "Yes"],
       ["Yellow", "Sports", "Domestic", "No"],
       ["Yellow", "Sports", "Imported", "Yes"],
       ["Yellow", "SUV", "Imported", "No"],
       ["Yellow", "SUV", "Imported", "Yes"],
       ["Yellow", "SUV", "Domestic", "No"],
       ["Red", "SUV", "Imported", "No"],
       ["Red", "Sports", "Imported", "Yes"]
       ]
   tree = build_tree(examples)
   print("Tree:")
   #print_tree(tree, feature_names)
   test_instance = ["Red", "SUV", "Domestic"]
   test_class = classify(tree, test_instance)
   print("\nTest instance: " + str(test_instance))
   print("  class = " + test_class)

Tree:
  Type == SUV
    Color == Red
      Stolen: No
    Color == Yellow
      Origin == Domestic
        Stolen: No
      Origin == Imported
        Stolen: Yes
  Type == Sports
    Origin == Domestic
      Color == Red
        Stolen: Yes
      Color == Yellow
        Stolen: No
    Origin == Imported
      Stolen: Yes

Test instance: ['Red', 'SUV', 'Domestic']
  class = No
